In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# import os
# py_file_location = "./drive/My Drive/LAB/COMP90086"
# sys.path.append(os.path.abspath(py_file_location))

# !pip install einops

In [1]:
from environment import *
from utils import *
from model import *

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# ROOT = './drive/My Drive/LAB/COMP90086/COMP90086_2023_TLLdataset 2'
ROOT = './COMP90086_2023_TLLdataset 2'
seed = 19260817
IMAGE_SIZE = 0
data_list = pd.read_csv(ROOT + '/train.csv')
future_list = pd.read_csv(ROOT + '/test_candidates.csv')

In [3]:
imgs = dict()
for img in tqdm(os.listdir(ROOT+'/train/left')):
  imgs[img] = prepare_image(ROOT+'/train/left/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/train/right')):
  imgs[img] = prepare_image(ROOT+'/train/right/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/test/left')):
  imgs[img] = prepare_image(ROOT+'/test/left/'+img, resize_shape=IMAGE_SIZE)

for img in tqdm(os.listdir(ROOT+'/test/right')):
  imgs[img] = prepare_image(ROOT+'/test/right/'+img, resize_shape=IMAGE_SIZE)

100%|██████████| 2000/2000 [00:02<00:00, 898.91it/s]


In [4]:
train_list, valtest_list = train_test_split(data_list, test_size=0.3, random_state=seed) # train is 70%
val_list, test_list = train_test_split(valtest_list, test_size=0.5, random_state=seed) # val is 15%, test is 15%

In [5]:
class CNN_Key_Query_Triplet_CFG:
    num_false               = 19 # how many hard negative to try for each anchor
    encoder                 = 'resnet18'
    freeze_encoder          = True
    input_shape             = (64, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else(64, 3, 245, 200)
    num_mlp_layers          = 2
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 128 # size of the actual embedding for each image
    pretrained              = True      
    crop_pretrained_linear  = True
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = 'resnet18_triplet'
    self_eval_batch_size    = 64

cnn_key_query_triplet = CNN_Key_Query_Triplet(CNN_Key_Query_Triplet_CFG)
cnn_key_query_triplet.fit(train_list, val_list, batch_size = 64, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_key_query_triplet.load()
print('Best Validation:')
cnn_key_query_triplet.eval(val_list, batch_size=128, return_loss=False)


print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_triplet.real_eval(future_test_list, 100)

final_out1 = cnn_key_query_triplet.real_eval(future_list, 100)

Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0
 10%|█         | 43/416 [11:50<1:42:39, 16.51s/it]


KeyboardInterrupt: 

In [30]:
class CNN_Key_Query_Categorisation_CFG:
    num_false               = 1 # how many false samples for each true sample
    target                  = 1 # target loss for correct. false will be 1-target
    encoder                 = 'resnet18' # the name of the model we want to load from torchhub
    freeze_encoder          = True
    input_shape             = (64, 3, IMAGE_SIZE, IMAGE_SIZE) if IMAGE_SIZE else(64, 3, 245, 200)  # (batch size, 3, [what our loaded in images look like])
    num_mlp_layers          = 2 # how many mlp layers in the end. 0 means no mlp and just 1 out layer; n means n mlp + outlayer
    hidden_dim              = 512 # hidden dim of mlp
    dropout                 = 0.1 # dropout rate of mlp
    res_learning            = False # whether to use residual layer in mlp
    pretrained              = True # whether to use pretrained encoder
    crop_pretrained_linear  = True
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss()   
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving models
    # rootpath                = py_file_location
    DataFactory_Categorisation = DataFactory_Categorisation
    DataLoader_Categorisation = DataLoader_Categorisation
    images                  = imgs
    name                    = 'resnet18_categorisation'
    self_eval_batch_size    = 64


cnn_key_query_categorisation = CNN_Key_Query_Categorisation(CNN_Key_Query_Categorisation_CFG)
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_categorisation.real_eval(future_test_list, 21)
cnn_key_query_categorisation.fit(train_list, val_list, batch_size = 64, epochs=100, patience=8, scheduler=True, grad_clip=False, mark='')

cnn_key_query_categorisation.load()
print('Best Validation:')
cnn_key_query_categorisation.eval(val_list, batch_size=128, return_loss=False)

print('Best Test:')
future_test_list = turn_val_into_future(test_list, seed)
cnn_key_query_categorisation.real_eval(future_test_list, 100)

final_out2 = cnn_key_query_categorisation.real_eval(future_list, 100)

Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0


(420,)
(21, 20)


KeyboardInterrupt: 